In [100]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
import json
from  tensorflow import keras
from tensorflow.keras import layers
import os
import tqdm as tqdm

In [101]:
shape =  np.stack(np.load('./processed_quickdraw/octopus.npy', allow_pickle = True)).shape

In [102]:

total_data = np.empty((0, shape[1], shape[2]))
y = np.array([])
#load  all datasets
input_folder = './small'
files = os.listdir(input_folder)
for idx, filename in enumerate(tqdm.tqdm(files, desc='Processing files', unit='file')):
    if filename.endswith(".npy"):
        data =  np.stack(np.load(os.path.join(input_folder, filename), allow_pickle = True))
        total_data = np.vstack((total_data, data))
        y = np.concatenate((y, np.ones(data.shape[0])*idx))
X = total_data
y = keras.utils.to_categorical(y)

Processing files: 100%|██████████| 12/12 [00:12<00:00,  1.07s/file]


In [103]:
y.shape

(12000, 12)

In [104]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [105]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.__version__)
print(tf.test.is_built_with_gpu_support())

Num GPUs Available:  1
2.10.0
True


In [106]:
# X = np.load('./processed_quickdraw/total_data.npy', allow_pickle= True)

In [107]:
X = X[:,:200,:]

In [108]:
X.shape

(12000, 200, 3)

In [109]:
X

array([[[ 0.02666667, -0.00987653,  0.        ],
        [ 0.01866665,  0.        ,  0.        ],
        [ 0.02933335, -0.00246914,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[-0.03614455, -0.02339183,  0.        ],
        [-0.03012049,  0.        ,  0.        ],
        [-0.03012049,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[-0.01532731, -0.00492644,  0.        ],
        [-0.02426508, -0.02167356,  0.        ],
        [-0.01404806, -0.0167495 ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       ...,

       [[-0.00643086, -0.01293901,  0.        ],
        [-0

In [110]:
X[:,199,2] = 1

In [111]:
X

array([[[ 0.02666667, -0.00987653,  0.        ],
        [ 0.01866665,  0.        ,  0.        ],
        [ 0.02933335, -0.00246914,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[-0.03614455, -0.02339183,  0.        ],
        [-0.03012049,  0.        ,  0.        ],
        [-0.03012049,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[-0.01532731, -0.00492644,  0.        ],
        [-0.02426508, -0.02167356,  0.        ],
        [-0.01404806, -0.0167495 ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       ...,

       [[-0.00643086, -0.01293901,  0.        ],
        [-0

In [112]:
y.shape

(12000, 12)

In [113]:
#setting up model
# model = keras.models.Sequential()
# model.add(keras.layers.Convolution1D(128,(6),activation='relu', input_shape=(200,3)))
# model.add(keras.layers.Convolution1D(64,(3),activation='relu'))
# model.add(keras.layers.Convolution1D(1, (1), activation='relu'))
# model.add(keras.layers.LSTM(128, return_sequences=True))
# # model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.LSTM(128, return_sequences=True))
# model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.LSTM(128, return_sequences=False))
# model.add(keras.layers.Dropout(0.2))
# model.add(Dense(y.shape[1]))
# # model.add(Dense(13))
# model.add(keras.layers.Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_33 (Conv1D)          (None, 195, 128)          2432      
                                                                 
 conv1d_34 (Conv1D)          (None, 193, 64)           24640     
                                                                 
 conv1d_35 (Conv1D)          (None, 193, 1)            65        
                                                                 
 lstm_27 (LSTM)              (None, 193, 128)          66560     
                                                                 
 lstm_28 (LSTM)              (None, 193, 128)          131584    
                                                                 
 dropout_10 (Dropout)        (None, 193, 128)          0         
                                                                 
 lstm_29 (LSTM)              (None, 128)             

In [114]:
y.shape

(12000, 12)

In [115]:
model = keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=X[0].shape))
model.add(tf.keras.layers.Conv1D(32, 3, activation='relu'))
model.add(tf.keras.layers.Conv1D(64, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(2))

model.add(tf.keras.layers.Conv1D(128, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(2))

# Recurrent layers (e.g., LSTM)
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LSTM(128))

# Dense layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))

# Output layer
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_36 (Conv1D)          (None, 198, 32)           320       
                                                                 
 conv1d_37 (Conv1D)          (None, 196, 64)           6208      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 98, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_38 (Conv1D)          (None, 96, 128)           24704     
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 48, 128)          0         
 g1D)                                                            
                                                                 
 lstm_30 (LSTM)              (None, 48, 128)         

In [116]:
X

array([[[ 0.02666667, -0.00987653,  0.        ],
        [ 0.01866665,  0.        ,  0.        ],
        [ 0.02933335, -0.00246914,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[-0.03614455, -0.02339183,  0.        ],
        [-0.03012049,  0.        ,  0.        ],
        [-0.03012049,  0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       [[-0.01532731, -0.00492644,  0.        ],
        [-0.02426508, -0.02167356,  0.        ],
        [-0.01404806, -0.0167495 ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ]],

       ...,

       [[-0.00643086, -0.01293901,  0.        ],
        [-0

In [117]:
y.shape

(12000, 12)

In [118]:
checkpoint = [keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/models.hdf5')]

In [119]:
mod = model.fit(X, y, epochs=100, batch_size= 100, callbacks= checkpoint, shuffle = True, validation_split=0.1)

Epoch 1/100


108/108 [==============================] - 5s 22ms/step - loss: 2.2940 - accuracy: 0.1794 - val_loss: 6.3802 - val_accuracy: 0.0000e+00
Epoch 2/100
108/108 [==============================] - 2s 16ms/step - loss: 2.1554 - accuracy: 0.2346 - val_loss: 6.8784 - val_accuracy: 0.0000e+00
Epoch 3/100
108/108 [==============================] - 2s 15ms/step - loss: 2.0938 - accuracy: 0.2556 - val_loss: 7.4278 - val_accuracy: 8.3333e-04
Epoch 4/100
108/108 [==============================] - 2s 15ms/step - loss: 1.9941 - accuracy: 0.3048 - val_loss: 7.8567 - val_accuracy: 0.0050
Epoch 5/100
108/108 [==============================] - 2s 14ms/step - loss: 1.8962 - accuracy: 0.3404 - val_loss: 8.4469 - val_accuracy: 0.0167
Epoch 6/100
108/108 [==============================] - 2s 15ms/step - loss: 1.7191 - accuracy: 0.4131 - val_loss: 8.9069 - val_accuracy: 0.0742
Epoch 7/100
108/108 [==============================] - 2s 15ms/step - loss: 1.5871 - accuracy: 0.4581 - val_loss: 9.0785 - val_accuracy:

KeyboardInterrupt: 

In [ ]:
mod = model.fit(X, y, epochs=100, batch_size= 100, callbacks= checkpoint, shuffle = True, validation_split=0.1)

Epoch 1/100


ValueError: in user code:

    File "c:\Users\Admin\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Admin\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Admin\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Admin\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Admin\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Admin\.pyenv\pyenv-win\versions\3.9.6\lib\sit

In [ ]:
total_data= np.array(total_data)

In [ ]:
pd.DataFrame(total_data)

ValueError: Must pass 2-d input. shape=(345, 1000, 655, 3)

In [ ]:
max_length = result_df.Strokes.apply(len).max()

padded_strokes = result_df.Strokes.apply(lambda x: x + [(0, 0, 0)] * (max_length - len(x)))

NameError: name 'result_df' is not defined

In [ ]:
#model construction
model = keras.Sequential()
model.add(layers.LSTM(64, input_shape=(None, 28)))
model.add(Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10))
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                23808     
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
Total params: 24714 (96.54 KB)
Trainable params: 24586 (96.04 KB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________
None
